### Callin Switzer

2 Nov 2016

-Read in all the Kalmia polygon images (from R KalmiaVisualizePolygons_V2.R)
- Make sure to use Python [conda env:vis_env]

-visualize in 3D

-Resample all of the images

-Stack resampled images (in 3D)

-Visualize a heatmap

-Save heatmap

In [1]:
import cv2

In [2]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

In [3]:
# scikit image
import skimage
from skimage import io

In [4]:
import os
import pandas as pd
import time

In [5]:
import scipy
import scipy.ndimage

from mpl_toolkits.mplot3d import Axes3D
import sys

In [6]:
from mayavi import mlab
from scipy import stats

In [7]:
% qtconsole

May want to look at this: http://stackoverflow.com/questions/25286811/how-to-plot-a-3d-density-map-in-python-with-matplotlib

In [18]:
# list files in directory

In [19]:
os.chdir("/Users/callinswitzer/Dropbox/KalmiaProject/KalmiaExamples/KalmiaManualTrig_VidsToProcess/")

In [20]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
photoList = [ x for x in onlyfiles if not x.startswith('.') and x.endswith('.png')]

In [ ]:
# read in image and resize
kk = 1
img = np.array(cv2.imread(photoList[kk], 0))

newSideLen = 200

r = float(newSideLen) / img.shape[1]
dim = (int(newSideLen), int(img.shape[0] * r))
 
# perform the actual resizing of the image and show it
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

resized = np.array((resized < 255) * 1.0)

# clear edges of resized image (grid from graph)
resized[:, 0:newSideLen/20] = 0
resized[newSideLen - newSideLen/20:newSideLen, :] = 0

plt.close('all')
io.imshow(resized)
thismanager = plt.get_current_fig_manager()
thismanager.window.wm_geometry("-1-1")




In [ ]:
plt.imshow(resized)

In [ ]:
# make empty np array
# for future reference, these are stacked wrong

# rotate image for visualization with numpy
resized_flip = np.rot90(resized, 3)
stackedArr = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0

midpt = stackedArr.shape[1]/2

stackedArr[midpt,:,:] = stackedArr[midpt,:,:] + resized_flip

stackedArr[:, midpt, :] = stackedArr[:, midpt, :] + resized_flip

# get size
sys.getsizeof(stackedArr)/1000/1000 # in MB


In [ ]:
x,y,z = stackedArr.nonzero()

In [ ]:
#plt.close('all')
plt.clf()
fig = plt.figure(1)
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, zdir='z', c = stackedArr[x,y,z], 
           cmap = 'Greys', edgecolor = 'none', marker = 's')
#plt.savefig("demo.png")

In [ ]:
plt.close('all')
mlab.close(all = True)

In [ ]:
# visualize with MayaVi points in 3d
x,y,z = stackedArr.nonzero()
xyz = np.vstack([x, y, z,])
kde = stats.gaussian_kde(xyz)
density = kde(xyz)

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')
pts = mlab.points3d(x, y, z, stackedArr[x,y,z], colormap = 'hot', 
                    scale_mode='none', scale_factor=1, opacity = 0.2)
mlab.axes()
mlab.show()

In [ ]:
# visualize density with MayaVi
density = stackedArr
grid = mlab.pipeline.scalar_field(density)
min1 = density.min()
max1=density.max()
mlab.pipeline.volume(grid, vmin=min1, vmax=min1 + .5*(max1-min1))

mlab.axes()
mlab.show()

# Rotate image

In [ ]:
plt.clf()
io.imshow(resized)

In [ ]:
# rotate image with opencv
w = resized.shape[0]
h = resized.shape[1]
M = cv2.getRotationMatrix2D((newSideLen/2,newSideLen/2), 36, 1.0)
rotated = cv2.warpAffine(resized, M, (w, h))
io.imshow(rotated < .4)


In [ ]:
# rotate with scipy
plt.clf()
i2 = scipy.ndimage.interpolation.rotate(np.array(resized), 36, axes=(1,0))
io.imshow(i2)

In [ ]:
stackedRot = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
stackedRot[midpt,:,:] = stackedRot[midpt,:,:] + resized_flip

In [ ]:
# visualize
mlab.contour3d(stackedRot)

In [ ]:
mlab.clf()
#pixels (should be an even number to make it symmetrical)

pollWidth = newSideLen / 100 / 2

stackedRot = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] = stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] + resized_flip



# pretty fast
w,h = stackedRot.shape[:2]
M = cv2.getRotationMatrix2D((newSideLen/2,newSideLen/2), 36, 1.0)
rotated = cv2.warpAffine(resized, M, (w, h))


for ii in range(newSideLen):
    stackedRot[:,:,ii] = ((cv2.warpAffine(stackedRot[:,:,ii], M, (w, h)) +
                          stackedRot[:,:,ii]) < 1 ) * 1
    
mlab.contour3d(stackedRot)
    

In [ ]:
stackedRot = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] = stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] + resized_flip


stackedRot_fin = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0

# do for all 10
for kk in range(10):
    M = cv2.getRotationMatrix2D((newSideLen/2,newSideLen/2), 36*kk, 1.0)

    for ii in range(newSideLen):
        stackedRot_fin[:,:,ii] = ((cv2.warpAffine(stackedRot[:,:,ii], M, (w, h)) > 0.3) * 1 +
                              stackedRot_fin[:,:,ii])

        


In [ ]:
# slice in z-direction
io.imshow(stackedRot_fin[:,:,newSideLen / 2])

In [ ]:
mlab.clf()
plt.close('all')
mlab.contour3d(stackedRot_fin, contours = 10)

In [ ]:
# final density visualization
mlab.clf()
density = stackedRot_fin
grid = mlab.pipeline.scalar_field(density)
min1 = density.min()
max1=density.max()
mlab.pipeline.volume(grid, vmin=min1, vmax=min1 + .5*(max1-min1))

mlab.axes()
mlab.show()

In [ ]:
# visualize with points
# visualize with MayaVi points in 3d
x,y,z = stackedRot_fin.nonzero()

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')
pts = mlab.points3d(x, y, z, stackedRot_fin[x,y,z], colormap = 'hot', 
                    scale_mode='none', scale_factor=1.5, opacity = 0.02)
mlab.axes()
mlab.show()

In [53]:
io.imshow(resized)
intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]]) / 10
plt.scatter(intPoints[:, 0], intPoints[:,1])

# Resampling and replotting

In [21]:
# read in all images in direcory, resize, and make a big numpy array
# read in image and resize

newSideLen = 200
img = np.array(cv2.imread(photoList[1], 0))
r = float(newSideLen) / img.shape[1]
dim = (int(newSideLen), int(img.shape[0] * r))

stackedImgs = np.ones((newSideLen, newSideLen, len(photoList)))


for hh in range(len(photoList)):
    img = np.array(cv2.imread(photoList[hh], 0)) # read in image black and white

    # perform the resizing
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    # convert to binary
    resized = np.array((resized < 255) * 1.0)

    # clear edges of resized image (grid from graph)
    resized[:, 0:newSideLen/20] = 0
    resized[newSideLen - newSideLen/20:newSideLen, :] = 0
    
    # add to stacked array
    stackedImgs[:,:,hh] = resized

In [22]:
def resampRotate(stackedImgs):
    '''
    Resample all the images (same number of photos that I took)
    Randomly rotate images in 1/10 of 360 degree increments
    Put into 3D space
    '''
    
    # prepare to resample resized images
    pollWidth = newSideLen / 100 / 2
    midpt = newSideLen / 2

    # make temporary array
    stackedRot = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
    w,h = stackedRot.shape[:2]


    # make final array
    stackedRot_fin = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0
    
    
    # resample all images
    photoSamp = np.random.choice(range(len(photoList)), size = len(photoList), replace = True)
    # resample rotation angles
    angles = np.random.choice(range(10), len(photoList), True)


    for kk in photoSamp:

        # get image to rotate
        resized = stackedImgs[:,:,kk]
        resized_flip = np.rot90(resized, 3)

        # get rotatation matrix
        M = cv2.getRotationMatrix2D((newSideLen/2,newSideLen/2), 36*angles[kk], 1.0)

        # reset blank 3D array
        stackedRot = stackedRot * 0.0

        # get image to rotate into 3D space
        stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] = stackedRot[midpt-pollWidth:midpt + pollWidth,:,:] + resized_flip

        # rotate image in 3D, by doing one level at a time
        for ii in range(newSideLen):
            stackedRot_fin[:,:,ii] = ((cv2.warpAffine(stackedRot[:,:,ii], M, (w, h)) > 0.3) * 1 +
                                  stackedRot_fin[:,:,ii])
        
    return stackedRot_fin

In [23]:
finArry = resampRotate(stackedImgs=stackedImgs)

mlab.contour3d(finArry)

In [24]:
# repeat the resampling method multiple times

res_fin = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0

stt = time.time()
for ii in range(500):
    tmp = resampRotate(stackedImgs)
    res_fin = res_fin + tmp
    print ii, time.time() - stt # takes about 8 seconds each

0 7.94893407822
1 15.6140861511
2 23.3463571072
3 30.7974450588
4 38.1466720104
5 45.4449059963
6 52.7925620079
7 60.1742119789
8 67.4737000465
9 74.8411369324
10 82.150591135
11 89.6021430492
12 96.9459140301
13 104.333492994
14 111.683259964
15 119.114547968
16 126.333704948
17 133.705682993
18 140.944761992
19 148.370623112
20 155.723433018
21 163.028471947
22 170.446948051
23 177.834399939
24 185.13595295
25 192.516236067
26 199.838824034
27 207.126547098
28 214.458611965
29 221.757526159
30 229.160057068
31 236.549175024
32 243.994274139
33 251.389869928
34 258.770826101
35 266.109580994
36 273.499665976
37 280.820874929
38 288.260019064
39 295.558576107
40 303.041866064
41 310.433721066
42 317.802962065
43 325.244544983
44 332.631605148
45 339.95599699
46 347.304616928
47 354.65841794
48 362.040907145
49 369.309448004
50 376.64786005
51 383.938364029
52 391.127247095
53 398.581810951
54 405.878926039
55 413.077055931
56 420.311195135
57 427.58112812
58 434.864010096
59 442.074779

In [25]:
# save numpy array, so I don't have to keep running the simulation
outfile = '/Users/callinswitzer/Desktop/array5.npy'
np.save(outfile, res_fin)

In [64]:
# load np array
res_fin = np.load('/Users/callinswitzer/Desktop/array5.npy')
mlab.close(all = True)
mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour3d(res_fin, contours= 30)

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]]) / 10
mlab.points3d([100, 100, 100], intPoints[:,0], 200 - intPoints[:,1], colormap = 'hot', 
                    scale_mode='none', scale_factor=5, opacity = 1)

In [12]:
mlab.close(all = True)
mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))

density = res_fin / np.max(res_fin)
grid = mlab.pipeline.scalar_field(density)
min1 = density.min()
max1=density.max()
mlab.pipeline.volume(grid, vmin=min1, vmax=min1 + .5*(max1-min1))
intPoints = (np.array([[768, 1694],[1000, 1694],[1231, 1694]]) / 10) + 0.5
mlab.points3d([100, 100, 100], intPoints[:,0], 200 - intPoints[:,1], colormap = 'hot', 
                    scale_mode='none', scale_factor=5, opacity = 1)

mlab.axes(extent = [25, 175, 25, 175, 15, 175])
#mlab.show()

In [78]:
mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))

density = res_fin / np.max(res_fin)
grid = mlab.pipeline.scalar_field(density)
min1 = density.min()
max1=density.max()
mlab.pipeline.volume(grid, vmin=min1, vmax=min1 + .5*(max1-min1))
intPoints = (np.array([[768, 1694],[1000, 1694],[1231, 1694]]) / 10) + 0.5
mlab.points3d([100, 100, 100], intPoints[:,0], 200 - intPoints[:,1], colormap = 'cool', 
                    scale_mode='none', scale_factor=5, opacity = 1)

mlab.axes(extent = [25, 175, 25, 175, 15, 175])
#mlab.show()

In [ ]:
# visualize with points
# visualize with MayaVi points in 3d
x,y,z = res_fin.nonzero()

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')
pts = mlab.points3d(x, y, z, res_fin[x,y,z], colormap = 'hot', 
                    scale_mode='none', scale_factor=1.5, opacity = 0.02)
mlab.axes()
mlab.show()

In [103]:
# this way is great!

mlab.close(all = True)
density = res_fin / np.max(res_fin)

fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour3d(density, contours= 50, opacity = 0.1, colormap = 'hot')
#mlab.axes(extent = [25, 175, 25, 175, 15, 175], y_axis_visibility = False)
mlab.orientation_axes()
mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
                    scale_mode='none', scale_factor=2, opacity = 1)

In [101]:
# this way is great!

mlab.close(all = True)
density = res_fin / np.max(res_fin)

fig = mlab.figure(size = (1024,768),
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
mlab.contour3d(density, contours= 50, opacity = 0.2, colormap = "jet")
#mlab.axes(extent = [25, 175, 25, 175, 15, 175], y_axis_visibility = False)
mlab.orientation_axes()
mlab.points3d([101, 101, 101], intPoints[:,0] + 1, 200 - intPoints[:,1], colormap = 'hot', 
                    scale_mode='none', scale_factor=2, opacity = 1)